In [1]:
from okx.app.candle_server import CandleRule,CandleServer

## 1. 重新定义规则

In [2]:
class MyCandleRule(CandleRule):
    # 收集的产品 all表示全部正在交易的产品
    INSTIDS = [
        'BTC-USD-SWAP',
        'ETH-USD-SWAP',
        'LTC-USD-SWAP',
        'XRP-USD-SWAP',
        'BCH-USD-SWAP',
        'SOL-USD-SWAP',
        'FIL-USD-SWAP',
        '1INCH-USD-SWAP',
        'ADA-USD-SWAP',
        'ALGO-USD-SWAP',
        'ATOM-USD-SWAP',
        'AVAX-USD-SWAP',
        'BSV-USD-SWAP',
        'CRV-USD-SWAP',
        'DASH-USD-SWAP',
        'DOGE-USD-SWAP',
        'DOT-USD-SWAP',
        'EOS-USD-SWAP',
        'ETC-USD-SWAP',
        'GRT-USD-SWAP'
    ]
    # candle_map 缓存K线数据长度
    LENGTH = 1440
    # 时间颗粒度
    BAR = '1m'
    # 每日下载昨日历史K线数据的时刻 格式：%H:%M:%S None表示不下载
    DOWNLOAD_TIME = None # 如果你维护的实时K线长度不足以包含一整天，应该关闭DOWNLOAD_TIME功能

## 2. 启动实时K线维护

In [3]:
# 币币交易：SPOT；永续合约：SWAP；交割合约：FUTURES；期权：OPTION
instType = 'SWAP'
# 永续合约，默认规则
candleServer = CandleServer(
    instType=instType, 
    rule=MyCandleRule,
    proxies = {},
    proxy_host='http://101.32.44.77:99',
    # 使用http和https代理，proxies={'http':'xxxxx','https:':'xxxxx'}，通requests中的proxies参数规则相同
    proxies={},
    # 转发：需搭建转发服务器，可参考：https://github.com/pyted/okx_resender
    proxy_host=None, 
)
# 启动K线维护服务（堵塞直到candle_map维护完成）
candleServer.run_candle_map()
# 被异步维护的candle_map字典
candleServer.candle_map

2023-10-25 21:12:36,430 [INFO] COMPLETE DOWNLOAD 2023-10-24 (ALL:20 SKIP:20 SUC:0 WARN:0 ERROR:0)
2023-10-25 21:12:36,440 [INFO] COMPLETE DOWNLOAD 2023-10-23 (ALL:20 SKIP:20 SUC:0 WARN:0 ERROR:0)
2023-10-25 21:12:46,212 [INFO] COMPLETE PREPARE candle_map
2023-10-25 21:12:47,147 [INFO] DOWNLOAD CANDLE CACHE


{'BTC-USD-SWAP': array([[1.69815312e+12, 3.43083000e+04, 3.44038000e+04, ...,
         4.37895000e+01, 1.50510000e+06, 1.00000000e+00],
        [1.69815318e+12, 3.43723000e+04, 3.43969000e+04, ...,
         1.52777400e+02, 5.25100000e+06, 1.00000000e+00],
        [1.69815324e+12, 3.43817000e+04, 3.44051000e+04, ...,
         6.02942000e+01, 2.07320000e+06, 1.00000000e+00],
        ...,
        [1.69823934e+12, 3.44941000e+04, 3.45884000e+04, ...,
         1.07408200e+02, 3.70880000e+06, 1.00000000e+00],
        [1.69823940e+12, 3.44507000e+04, 3.44909000e+04, ...,
         1.76539000e+01, 6.08300000e+05, 1.00000000e+00],
        [1.69823946e+12, 3.44865000e+04, 3.44865000e+04, ...,
         5.25680000e+00, 1.81200000e+05, 1.00000000e+00]]),
 'ETH-USD-SWAP': array([[1.69815312e+12, 1.82187000e+03, 1.82887000e+03, ...,
         3.23720600e+02, 5.91240000e+05, 1.00000000e+00],
        [1.69815318e+12, 1.82639000e+03, 1.82738000e+03, ...,
         1.10663900e+02, 2.02120000e+05, 1.00000000

## 3. 模拟实盘交易实时获取K线

In [4]:
import time
import datetime

# 打印实时更新的candle
for i in range(6):
    # 以安全的方式从candle_map中获取历史线，K线最新时间不能小于当前时间戳大于security_seconds秒
    candle = candleServer.get_candle_security('BTC-USD-SWAP',security_seconds=60 * 3)
    print(
        'K线长度:',
        candle.shape[0],
        'K线的最新时间:',
        datetime.datetime.fromtimestamp( int(candle[-1,0] / 1000) ),
        '当前时间:',
        datetime.datetime.now()
    )
    print(candle)  # 类型：np.ndarray
    time.sleep(10)

K线长度: 1440 K线的最新时间: 2023-10-25 21:11:00 当前时间: 2023-10-25 21:12:47.246713
[[1.69815312e+12 3.43083000e+04 3.44038000e+04 ... 4.37895000e+01
  1.50510000e+06 1.00000000e+00]
 [1.69815318e+12 3.43723000e+04 3.43969000e+04 ... 1.52777400e+02
  5.25100000e+06 1.00000000e+00]
 [1.69815324e+12 3.43817000e+04 3.44051000e+04 ... 6.02942000e+01
  2.07320000e+06 1.00000000e+00]
 ...
 [1.69823934e+12 3.44941000e+04 3.45884000e+04 ... 1.07408200e+02
  3.70880000e+06 1.00000000e+00]
 [1.69823940e+12 3.44507000e+04 3.44909000e+04 ... 1.76539000e+01
  6.08300000e+05 1.00000000e+00]
 [1.69823946e+12 3.44865000e+04 3.44865000e+04 ... 5.25680000e+00
  1.81200000e+05 1.00000000e+00]]
K线长度: 1440 K线的最新时间: 2023-10-25 21:11:00 当前时间: 2023-10-25 21:12:57.251034
[[1.69815312e+12 3.43083000e+04 3.44038000e+04 ... 4.37895000e+01
  1.50510000e+06 1.00000000e+00]
 [1.69815318e+12 3.43723000e+04 3.43969000e+04 ... 1.52777400e+02
  5.25100000e+06 1.00000000e+00]
 [1.69815324e+12 3.43817000e+04 3.44051000e+04 ... 6.029

## 4. 关闭服务

In [5]:
# 关闭服务
candleServer.close_run_candle_map()